# SPMF - Sequence-aware recommender system
- [SPMF](http://philippe-fournier-viger.com/spmf/index.php?link=download.php)

In [1]:
import numpy as np
import warnings
import pandas as pd
from spmf import Spmf
from gensim.models import Word2Vec
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils import np_utils
from keras.regularizers import l2
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display

c:\app\anaconda3\envs\yjformat\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\app\anaconda3\envs\yjformat\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\app\anaconda3\envs\yjformat\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
YJPATH3 = r'F:/YJ-format-230125/data/'; OUTDIR3 = r'interim/'
df = pd.read_csv(YJPATH3 + OUTDIR3 + r'pbp-zone-2023-03-20.csv', encoding='cp949', low_memory=False)
df = df.sort_values(by=['gid-intot-panum-pitnum'], ascending=True)
def one_digit(n): return f"{n:.0f}"
def two_digit(n): return f"{n:02d}"
def three_digit(n): return f"{n:03d}"
def six_digit(n): return f"{n:06d}"
df['zone101_new']=df['zone101_new'].apply(two_digit)
df['bs_cnt']=df['bs_cnt'].apply(two_digit)
df['bso_cnt']=df['bso_cnt'].apply(three_digit)
df['bso_rcnt']=df['bso_rcnt'].apply(six_digit)
pit_len = df.groupby(['gid-intot-panum']).agg(pit_len=('pitch_number','last')).reset_index()
dfdf = pd.merge(df, pit_len, on='gid-intot-panum', how='left')
dfdf['pa_cate_en3'] = dfdf['pa_cate_en3'].bfill()

cf=dfdf[(dfdf['pit_len']>=3)][['cid', 'game_date', 'gid-intot-panum', 'pitch_type', 'pitch_number', 'bs_cnt', 'bso_cnt', 'pit_len']]

cf['gid-intot-panum-cid'] = cf['gid-intot-panum']+'-'+cf['cid']
cf["game_date"] = pd.to_datetime(cf.game_date, format='%Y-%m-%d')
data = cf[['gid-intot-panum-cid', 'game_date', 'pitch_type']].copy()

In [22]:
# cf=dfdf[(dfdf['pit_len']>=3)&(dfdf['yy']>=2020)&(dfdf['cid']=='양의지-76232')][['cid','gid', 'game_date', 'gid-intot', 'gid-intot-panum', 'pitch_type', 'pitch_number', 'bs_cnt', 'bso_cnt', 'pit_len', 'pitch_type_le']]
cf=dfdf[(dfdf['pit_len']>=3)][['cid','gid', 'game_date', 'gid-intot', 'gid-intot-panum', 'pitch_type', 'pitch_number', 'bs_cnt', 'bso_cnt', 'pit_len', 'pitch_type_le']]
cf['gid-intot-panum-cid'] = cf['gid-intot-panum']+'-'+cf['cid']
cf['gid-intot-cid'] = cf['gid-intot']+'-'+cf['cid']
cf['gid-cid'] = cf['gid']+'-'+cf['cid']
cf["game_date"] = pd.to_datetime(cf.game_date, format='%Y-%m-%d')
data = cf[['gid-cid', 'gid-intot-cid', 'gid-intot-panum-cid', 'game_date', 'pitch_type_le']].copy()
data['pitch_type_le'] = data['pitch_type_le'].astype(str)

In [24]:
dfdf.groupby(['pitch_type_le','pitch_type'])[['gid-intot-panum-pitnum']].nunique().reset_index().T

,0,1,2,3,4,5,6,7,8
pitch_type_le,1,2,3,4,5,6,7,8,9
pitch_type,너클,슬라,싱커,직구,체인,커브,커터,투심,포크
gid-intot-panum-pitnum,1219,242405,170,587378,134669,125122,31741,100644,84873


### 0. 패턴용 데이터 만들기
- 중간구분자 = -1
- 줄바꿈구분자 = -2

In [25]:
transactions_array = []
# data_new = data.groupby(['gid-cid', 'gid-intot-cid', 'gid-intot-panum-cid']).agg({'pitch_type_le': lambda x: ' '.join(x)}).reset_index()
# transactions = data_new.groupby(['gid-intot-cid'])['pitch_type_le'].apply(list)
transactions = data.groupby(['gid-intot-panum-cid'])['pitch_type_le'].apply(list)
for i in range(0, len(transactions)):
    vals = ' -1 '.join(transactions.values[i])
    transactions_array.append(vals)
transactions_array[:10]

['4 -1 4 -1 4 -1 4 -1 4',
 '2 -1 2 -1 2',
 '4 -1 6 -1 4 -1 5 -1 4 -1 2',
 '6 -1 4 -1 4 -1 5 -1 6',
 '4 -1 6 -1 4 -1 4 -1 2 -1 8 -1 2',
 '4 -1 4 -1 4',
 '4 -1 4 -1 4',
 '4 -1 4 -1 4 -1 2',
 '2 -1 4 -1 8 -1 4',
 '4 -1 4 -1 4 -1 2 -1 5']

In [26]:
with open('input_pitch.txt', 'w') as f:
    for line in transactions_array:
        f.write(f"{line} -2\n")

### 1. GSP: Generalized Sequential Pattern
-  GSP algorithm: Apriori 기반

In [20]:
result_GSP = Spmf("GSP", input_filename="input_pitch.txt", arguments=[0.02]
                  , output_filename='GSP.txt', spmf_bin_location_dir="F:/Recommender/SequenceAwareRecommenderSystem"
                  )
result_GSP.run()
df = result_GSP.to_pandas_dataframe(pickle=True)
df = df[df.pattern.str.len() > 2]
df.sort_values('sup', ascending=False).head(20)

>/F:/Recommender/SequenceAwareRecommenderSystem/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 157 ms
 Frequent sequences count : 87
 Max memory (mb):87.97164154052734




,pattern,sup
52,"[4, 4, 4]",1268
51,"[4, 4, 2]",480
47,"[2, 4, 4]",466
50,"[4, 2, 4]",466
84,"[4, 4, 4, 4]",428
49,"[4, 2, 2]",340
44,"[2, 2, 2]",335
45,"[2, 2, 4]",328
56,"[4, 4, 9]",326
46,"[2, 4, 2]",293


### 2. PrefixSpan
- 후보 패턴을 만들지 않으면서 빈번한 패턴을 찾는 방법
-  PrefixSpan Tree를 만들어가며 빈번한 패턴 탐색

In [30]:
prefixSpan = Spmf("PrefixSpan", input_filename="input_pitch.txt", arguments=[0.02]
            , output_filename="PrefixSpan.txt", spmf_bin_location_dir="F:/Recommender/SequenceAwareRecommenderSystem")
prefixSpan.run()
df = prefixSpan.to_pandas_dataframe(pickle=True)
df = df[df.pattern.str.len() > 3]
df.sort_values('sup', ascending=False).head(30)
# df_prefixSpan.to_csv("output.csv")

>/F:/Recommender/SequenceAwareRecommenderSystem/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 436 ms
 Frequent sequences count : 132
 Max memory (mb) : 139.25096893310547
 minsup = 4909 sequences.
 Pattern count : 132




,pattern,sup
46,"[4, 4, 4, 4]",44438
48,"[4, 4, 4, 4, 4]",17779
44,"[4, 4, 4, 2]",17323
41,"[4, 4, 2, 4]",16465
34,"[4, 2, 4, 4]",16044
15,"[2, 4, 4, 4]",15672
40,"[4, 4, 2, 2]",11624
7,"[2, 2, 4, 4]",11262
31,"[4, 2, 2, 4]",11234
33,"[4, 2, 4, 2]",10920


### 3.SPADE
- Sequential pattern mining in Vertical Format Database

In [31]:
SPADE = Spmf("SPADE", input_filename="input_pitch.txt", arguments=[0.02]
            , output_filename="SPADE.txt", spmf_bin_location_dir="F:/Recommender/SequenceAwareRecommenderSystem")
SPADE.run()
df = SPADE.to_pandas_dataframe(pickle=True)
df = df[df.pattern.str.len() > 2]
df.sort_values('sup', ascending=False).head(20)

>/F:/Recommender/SequenceAwareRecommenderSystem/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 624 ms
 Frequent sequences count : 132
 Join count : 336
 Max memory (mb):428.49656677246094
Content at file SPADE.txt




,pattern,sup
80,"[4, 4, 4]",92456
94,"[4, 4, 4, 4]",44438
69,"[4, 4, 2]",39676
70,"[4, 2, 4]",38327
121,"[2, 4, 4]",37715
68,"[4, 2, 2]",29082
114,"[2, 2, 4]",28841
113,"[2, 4, 2]",26895
112,"[2, 2, 2]",26389
44,"[6, 4, 4]",21471


### 4. CloSpan

In [33]:
CloSpan = Spmf("CloSpan", input_filename="input_pitch.txt", output_filename="ColSpan.txt"
            , arguments=[0.02], spmf_bin_location_dir="F:/Recommender/SequenceAwareRecommenderSystem")
CloSpan.run()
df = CloSpan.to_pandas_dataframe(pickle=True)
df = df[df.pattern.str.len() > 3]
df.sort_values('sup', ascending=False).head(20)

>/F:/Recommender/SequenceAwareRecommenderSystem/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 2341 ms
 Frequent sequences count : 83
 Max memory (mb):499.3688278198242
Content at file ColSpan.txt





,pattern,sup
42,"[4, 4, 4, 4]",24776
39,"[4, 4, 2, 4]",8932
41,"[4, 4, 4, 2]",8928
32,"[4, 2, 4, 4]",8644
14,"[2, 4, 4, 4]",8225
43,"[4, 4, 4, 4, 4]",8114
29,"[4, 2, 2, 4]",6021
8,"[2, 2, 4, 4]",6016
38,"[4, 4, 2, 2]",5744
31,"[4, 2, 4, 2]",5268
